### MEMO_0901 (v0901_1.0)

### _* Memo_0901_3_

In [24]:
## Configuration

version="memo_0901_3"
next_round = 1188

title = "Metrics"
row_length = 400
new_to_old = True
show_mark_down = True
markdown_table_limit_length = 20
op_with_re = False
db_path = '../db/metrics.db'

In [25]:
## this code was run.
#!pip install keras-tuner
# import numpy as np
# import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
# print(f'Current Step #01 [now = {datetime.now()}')
from IPython.display import Markdown
import sys
sys.path.append("/home/jovyan/work/LSTM_Analyze/foresttrans")
from lib.analy_util import ( draw_markdown,
                             draw_graph,
                             draw_scatter,
                             get_pre_bef,
                             draw_average,
                             get_frequency,
                             get_information,
                             analyze_v1,
                             data_prepare,
                             search_model,
                             get_information_v2,
                             get_extdatas,
                             )
print(f'define this code. [{datetime.now()}]')


define this code. [2025-09-01 18:41:29.533700]


In [33]:
def get_candidated(version: str):
    """ get_candidated """
    query = f'select n0, n1, n2, n3, n4, n5 from candidated where version=\\"{version}\\";'
    metrics = !echo "{query}" | sqlite3 {db_path}
    print('metrics = ', metrics)
    if len(metrics) == 1:
        metrics = metrics[0].split('|')
        if len(metrics) == 6:
            return {0: int(metrics[0]),
                    1: int(metrics[1]),
                    2: int(metrics[2]),
                    3: int(metrics[3]),
                    4: int(metrics[4]),
                    5: int(metrics[5])}
    return {}


def set_candidated_level(version: str, level: int):
    """ set_candidated_level """
    """ candidated의 level을 설정합니다. """ 
    up_query = f'update candidated set level={level} where version=\\"{version}\\";'
    !echo "{up_query}" | sqlite3 {db_path}


def get_candidated_level(version: str):
    """ get_candidated_level """
    """ level을 가져 옵니다. """
    sel_query = f'select level from candidated where version=\\"{version}\\";'
    levels = !echo "{sel_query}" | sqlite3 {db_path}

    if len(levels) > 0:
        return int(levels[0])
    return 0


def update_candidated(candidated: dict, version: str):
    """ update_candidated """
    sel_query = f'select * from candidated where version=\\"{version}\\";'
    metrics = !echo "{sel_query}" | sqlite3 {db_path}
    if len(metrics) == 1:
        """ update """
        update_query = 'update candidated set '
        set_kv = []
        for key in candidated.keys():
            set_kv.append(f'n{key}={candidated[key]}')
        update_query += ", ".join(set_kv)
        update_query += f' where version=\\"{version}\\";'
        !echo "{update_query}" | sqlite3 {db_path}
    else:
        """ insert """
        insert_query = 'insert into candidated ('
        new_key = []
        new_val = []
        for key in candidated.keys():
            new_key.append(f'n{key}')
            new_val.append(f'{candidated[key]}')
        new_key.append('version')
        new_val.append(f'\\"{version}\\"')
        insert_query += f'{", ".join(new_key)} ) values ('
        insert_query += f'{", ".join(new_val)} );'
        !echo "{insert_query}" | sqlite3 {db_path}


def create_candidated():
    """ create_candidated """
    new_query = 'create table candidated ( ' \
                ' id INTEGER PRIMARY KEY AUTOINCREMENT, ' \
                ' version char[14], ' \
                ' n0 int, ' \
                ' n1 int, ' \
                ' n2 int, ' \
                ' n3 int, ' \
                ' n4 int, ' \
                ' n5 int );'
    !echo "{new_query}" | sqlite3 {db_path}


def draw_nums(ext_datas, is_pre_data=False, fig_size=(15, 8)):
    """ draw_nums """
    show_cols = [0, 1, 2, 3, 4, 5]
    show_label = False
    show_cnt = 100
    start_pos = row_length - show_cnt
    show_data = []
    for i in range(6):
        end_pos = -1 if is_pre_data else len(row_list_none_last[i])
        show_data.append(row_list_none_last[i][start_pos:end_pos])
    print(ext_datas)
    draw_scatter(show_data, f'line num : {key}', True, True, show_cols, ext_datas, show_label, helper_line_nums=[10, 20, 30, 40], fig_size=fig_size)


# 기준이 되는 수에 대한 DB의 기존 데이터를 출력합니다.
def get_likey(like_query, next_round):
    """ get_likey """
    rows = !echo "{like_query}" | sqlite3 {db_path}
    cur_next_round = next_round
    if len(rows) > 0:
        for row in rows:
            cols = row.split("|")
            nums = [int(i) for i in cols[2].split(',')]
            nums_str = [f"{int(i): >2}" for i in nums]
            round_diff = 0
            cur_round = int(cols[1])        
            if next_round > 0:
                round_diff = cur_next_round - cur_round
            cur_next_round = cur_round
            print(f"{cur_round: >4} | {','.join(nums_str)} | {sum(nums): >3} | {round_diff: >3}")
    else:
        print('data is empty.')

print('current work', datetime.now())
# create_candidated()
print(f'code defined {datetime.now()}')
cur_level = get_candidated_level(version=version)

current work 2025-09-01 18:46:31.168291
code defined 2025-09-01 18:46:31.168424


In [28]:
## 레벨을 설정합니다.
## 변경시에는 아래의 cur_level을 해제하고 실행하여야 합니다.
db_level = get_candidated_level(version=version)
print(f'cur_level = {cur_level}, db_level = {db_level}')
if cur_level != db_level:
    set_candidated_level(version=version, level=cur_level)
    db_level = get_candidated_level(version=version)
    print(f'cur_level = {cur_level}, db_level = {db_level}')
cur_level = 2

cur_level = 2, db_level = 0
cur_level = 2, db_level = 2


In [34]:
candidate = get_candidated(version=version)
if len(candidate.keys()) == 0:
    candidate = {0:1, 1:13, 2:21, 3:25, 4:28, 5:31}
print('candidate = ', candidate)


metrics =  ['7|12|24|30|34|']


ValueError: invalid literal for int() with base 10: ''

In [17]:
query = 'select round, metric, bonus from results'
query += f' order by round desc '
query += f'limit 1'
rows = !echo "{query}"| sqlite3 {db_path}
cols = rows[0].split('|')
latest_nums = [int(i) for i in cols[1].split(',')]

print('latest_nums = ', latest_nums)

latest_nums =  [5, 13, 26, 29, 37, 40]


In [ ]:
# Display the variable's value in Markdown
contents = f"###### Updated {datetime.now()}\n\n### {title}\n"
query = 'select round, metric, bonus from results'
query += f' order by round {"desc" if new_to_old else "asc"}'
limit_str = f'limit {row_length}'
query += f' {limit_str if row_length > 0 else ""}'
rows = !echo "{query}"| sqlite3 {db_path}
table_rows = ["|round|numbers|bonus|sum|0|10|20|30|40|",
              "|--|--|--|--|--|--|--|--|--|"
             ]
rows_dict = {1:[], 10:[], 20:[], 30:[], 40:[]}
nums_t = []
sums = []
cur_table_limit = 0
print(f'row\'s len = {len(rows)}')
candidate_strs = [f"{i}" for i in candidate.values()]
skip_first = True
for row in rows:
    if skip_first:
        row = f"{next_round}|{",".join(candidate_strs)}|0"
        skip_first = False
    cols = row.split('|')
    if len(cols) == 3:
        num_00 = 0
        num_10 = 0
        num_20 = 0
        num_30 = 0
        num_40 = 0
        nums_list = [int(i) for i in cols[1].split(',')]
        for num in nums_list:
            if num < 10:
                num_00 += 1
            if num >= 10 and num < 20:
                num_10 += 1
            if num >= 20 and num < 30:
                num_20 += 1
            if num >= 30 and num < 40:
                num_30 += 1
            if num >= 40 :
                num_40 += 1
        rows_dict[1].append(num_00)
        rows_dict[10].append(num_10)
        rows_dict[20].append(num_20)
        rows_dict[30].append(num_30)
        rows_dict[40].append(num_40)
        if cur_table_limit < markdown_table_limit_length:
            nums=[f"{i:02}" for i in nums_list] 
            table_rows.append(f"|{cols[0]}|{",".join(nums)}|{cols[2]}|{sum(nums_list)}|{num_00}|{num_10}|{num_20}|{num_30}|{num_40}|")
            cur_table_limit += 1
        nums_t.append(nums_list)
        sums.append(sum(nums_list))
sums.reverse()
table_rows_str = "\n".join(table_rows)
contents+=table_rows_str + "\n" + f"###### Updated {datetime.now()}"
Markdown(contents)

In [ ]:
# 기준이 되는 수에 대한 DB의 기존 데이터를 출력합니다.
print(f'Round|Metrics{" "*12}|Sum  |Width\n{"-"*40}')

like_part = '%,13,17,%,%,%'

### 선택번호와 비슷한 영역의 데이터 출력
like_query = f'select * from results where metric like \\"{like_part}\\" order by round desc;'
get_likey(like_query, next_round)

print(f'\nSelected Data\n')
print(f'Round|Metrics{" "*12}|Sum  |Width\n{"-"*40}')

### 선택 데이터 출력
candid_strs = [f"{i: >2}" for i in candidate.values()]
print(f'1185 | {",".join(candid_strs)} | {sum(list(candidate.values())): >3} |   0')
previous_round = (next_round-2, next_round-1)
previous_query = f'select * from results where round in {previous_round} order by round desc;'
get_likey(previous_query, next_round)

### 선택 번호와 비슷한 영역의 데이터 앞 출력
like_query_2 = f'select round from results where metric like \\"{like_part}\\" order by round desc;'
rounds = !echo "{like_query_2}" | sqlite3 {db_path}
wanted_rounds = []
for round in rounds:
    round = int(round)
    wanted_rounds.append((round-2, round-1, round))

for wanted_round in wanted_rounds:
    print(f"\n{"-"*40}")
    range_query = f'select * from results where round in {wanted_round} order by round desc;'
    get_likey(range_query, next_round)


In [ ]:
"""
data preparation
"""

#########
# nums_t
nums_by_row = [[] for i in range(6)]
for nums in nums_t:
    for i in range(6):
        nums_by_row[i].append(nums[i])
for i in range(6):
    nums_by_row[i].reverse()

##########
# row_list
row_list = []
row_list_none_last = []
for nums in nums_by_row:
    row_list.append(nums[0:-1])
    row_list_none_last.append(nums[0: len(nums)-1])

In [31]:
ext_datas = []

for key in candidate:
    ext_datas.append((key, candidate[key]))

# show_cols = [0, 1, 2, 3, 4, 5]
show_cols = [0, 1, 2, 3, 4, 5]
show_label = False
show_cnt = 150

start_pos = row_length - show_cnt

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, show_cnt // 25 +1):
    vline_poses.append((25*i, 0, 45))
vline_poses.append((68, 0, 45))
vline_poses.append((69, 0, 45))

draw_scatter(Y=show_data,
             title=f'Candidated Nums : {latest_nums} / {list(candidate.values())}',
             show_line=True,
             show_dot=True,
             show_cols=show_cols,
             ext_datas=ext_datas,
             show_label=show_label,
             helper_line_nums=[1, 3, 10, 14, 20, 28, 30, 35, 40, 45],
             marker_size=3,
             fig_size=(12,6),
             save_fig=False,
             show_time=False,
             vline_poses=vline_poses,
             vline_color="random",
            )
print(f'execute this code. [{datetime.now()}]')
# 10, 11, 14, 18, 27, 37

NameError: name 'candidate' is not defined

In [7]:
candidate

NameError: name 'candidate' is not defined

In [8]:
start_pos = 250
s1 = sums[start_pos:-1] + [sum(candidate.values())]
draw_scatter([s1], f'Sums', helper_line_nums=[60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160], show_vline=True)
print('candidate = ', sum(candidate.values()))
print('candidate = ', candidate.values())
print('last      = ', sums[:-1])
print(f'execute this code. [{datetime.now()}]')

NameError: name 'sums' is not defined

In [32]:
Round = 1
############################
selected = Round - 1
ext_data = 7

if op_with_re:
    ext_data = candidate[selected]

before = 0
after = candidate[selected+1]

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 30
helper_line_nums_1 = [0, 1, 4, 8, 10, 13, 15, 20, 25, 30, 35]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-2, -1, 0, 1, 2, 3, 4, 5, 10]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 150
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

NameError: name 'candidate' is not defined

In [10]:
Round = 2
############################
selected = Round - 1

ext_data = 12

if op_with_re:
    ext_data = candidate[selected]

before = candidate[selected-1]
after = candidate[selected+1]

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 30
helper_line_nums_1 = [0, 4, 8, 10, 13, 15, 20, 25, 30, 35]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-4, -3, -2, -1, 0, 3, 5, 7, 9, 11]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 150
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

NameError: name 'candidate' is not defined

In [11]:
Round = 3
############################
selected = Round - 1

ext_data = 24

if op_with_re:
    ext_data = candidate[selected]
    
before = candidate[selected-1]
after = candidate[selected+1]

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 30
helper_line_nums_1 = [0, 4, 8, 10, 13, 15, 20, 25, 30, 35]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-4, -3, -2, -1, 0, 3, 5, 7, 9, 11]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 150
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

NameError: name 'candidate' is not defined

In [ ]:
Round = 4
############################
selected = Round - 1

ext_data = 30

if op_with_re:
    ext_data = candidate[selected]

before = candidate[selected-1]
after = candidate[selected+1]

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 30
helper_line_nums_1 = [10, 13, 15, 20, 25, 30, 35, 40, 45]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-4, -3, -2, -1, 0, 3, 5, 7, 9, 11]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 180
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

In [ ]:
Round = 5
############################
selected = Round - 1

ext_data = 34

if op_with_re:
    ext_data = candidate[selected]

before = candidate[selected-1]
after = candidate[selected+1]

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 50
helper_line_nums_1 = [10, 12, 15, 20, 25, 28, 30, 33, 35, 40, 45]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-4, -3, -2, -1, 0, 3, 5, 7, 9, 11]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 150
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

In [35]:
Round = 6
############################
selected = Round - 1

ext_data = 38

if op_with_re:
    ext_data = candidate[selected]

before = candidate[selected-1]
after = 46

print(f"before   = {before}")
print(f"ext_data = {ext_data}")
print(f"after    = {after}")

wanted_data_length = 30
helper_line_nums_1 = [17, 20, 25, 30, 35, 40, 44, 45]
# helper_line_nums_2 = [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 7, 9, 11]
helper_line_nums_2 = [-4, -3, -2, -1, 0, 3, 5, 7, 9, 11]
# lengths = [2, 5, 8, 10, 13, 15, 20, 25, 30]
lengths = [10]
start_pos_0 = 150
start_pos_1 = 250
num_data=row_list_none_last[selected]
show_after = False
show_before = False
show_average = True
min_max_last_num_length = 20
show_diff_nums = False

ext_datas = get_extdatas(show_before, show_after, before, ext_data, after)

show_data = []
for i in range(6):
    show_data.append(row_list[i][start_pos:])

vline_poses = []
for i in range(1, (len(row_list_none_last[selected]) - start_pos_0) // 50 +1):
    vline_poses.append((50*i, 0, 45))

get_information_v2(selected=selected,
                   last_results=row_list_none_last,
                   ext_datas=ext_datas,
                   wanted_data_length=wanted_data_length,
                   helper_line_nums_1=helper_line_nums_1,
                   helper_line_nums_2=helper_line_nums_2,
                   show_diff=True,
                   lengths=lengths,
                   start_pos_0=start_pos_0,
                   start_pos_1=start_pos_1,
                   show_after=show_after,
                   show_before=show_before,
                   show_average=show_average,
                   min_max_last_num_length=min_max_last_num_length,
                   show_diff_nums=show_diff_nums,
                   vline_poses=vline_poses
                   )
candidate[selected] = ext_data
update_candidated(candidated={selected: ext_data}, version=version)
print(f'execute this code. [{datetime.now()}]')

NameError: name 'candidate' is not defined

In [ ]:
print(candidate)

In [21]:
print('now = ', datetime.now())

now =  2025-09-01 13:54:53.116287
